# SEIRHVD model example
## Work in progress (equations not ready)
\begin{align}
\dot{S} & = S_f - \alpha\beta\frac{SI}{N+k_I I+k_R R} + r_{R\_S} R\\
\dot{E} & = E_f + \alpha\beta\frac{SI}{N+k_I I+k_R R} - E\frac{1}{t_{E\_I}} \\
\dot{I} & = I_f + E\frac{1}{t_{E\_I}} - I\frac{1}{t_{I\_R}} \\
\dot{R} & = R_f + I\frac{1}{t_{I\_R}} - r_{I\_R} R\\
\end{align}
Where:  
* $S:$ Susceptible
* $E:$ Exposed
* $I:$ Infectious
* $R:$ Removed
* $\alpha:$ Mobilty
* $\beta:$ Infection rate
* $N:$ Total population
* $t_{E\_I}:$ # Transition time between exposed and infectious
* $t_{I\_R}:$ # Transition time between infectious and recovered
* $r_{R\_S}:$ Immunity loss rate ($\frac{1}{t_{R\_S}}$)  
* $S_f,E_f,I_f,R_f:$ External flux
* $k_I:$ Infected saturation   
* $k_R:$ Immunity shield    


In [65]:
# Util libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
# Adding lib paths
import sys
sys.path.insert(1, '../src2/models/')
sys.path.insert(1, '../src2/utils/')
sys.path.insert(1, '../src2/data/')

# cv19 libraries
from SEIRHVD import SEIRHVD 
from SEIR import SEIR 
import cv19functions
from cv19data import ImportData

#sys.path.insert(1, '../src2/data/')
#sys.path.insert(1, '../src2/utils/')
#import cv19functions
#import cv19data

In [66]:
# For pop-up plots execute this code (optional)
import platform
OS = platform.system()

if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
elif OS == 'Darwin':
    %matplotlib tk
    print('Mac (Funciona?)')

Linux


# Build a basic SEIR model using a configuration file

## Constant parameters

In [67]:
import importlib

In [68]:
importlib.reload(cv19functions)
import SEIRHVD
importlib.reload(SEIRHVD)
from SEIRHVD import SEIRHVD


In [69]:
# Input configuration file
config = 'cfg/SEIRHVD_SEIR.toml'
# Build simulation object
model = SEIRHVD(config = config)

In [70]:
%%capture
# Simulate (solve ODE)
model.integrate()

In [71]:
# Show results
model.results

,t,dates,S,Sv,E,E_d,Ev,Ev_d,Im,Im_d,...,Iv_ac_det,I,I_d,I_ac,I_det,I_d_det,I_ac_det,prevalence_total,prevalence_susc,prevalence_det
0,0,None,986000,0,3000,1500,0,0,6000,2999,...,0,6000,2999,18000,6000,2999,18000,0.018000,0.018000,0.018000
1,1,None,984813,0,3531,1302,0,0,6052,1523,...,0,6052,1523,19523,6052,1523,19523,0.019523,0.019523,0.019523
2,2,None,983609,0,3982,1241,0,0,6193,1040,...,0,6193,1040,20563,6193,1040,20563,0.020564,0.020564,0.020564
3,3,None,982372,0,4382,1240,0,0,6401,916,...,0,6401,916,21479,6401,916,21479,0.021480,0.021480,0.021480
4,4,None,981090,0,4750,1269,0,0,6662,918,...,0,6662,918,22398,6662,918,22398,0.022398,0.022398,0.022398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,None,201828,0,0,0,0,0,0,0,...,0,0,0,806823,0,0,806823,0.806824,0.806824,0.806824
496,496,None,201828,0,0,0,0,0,0,0,...,0,0,0,806823,0,0,806823,0.806824,0.806824,0.806824
497,497,None,201828,0,0,0,0,0,0,0,...,0,0,0,806823,0,0,806823,0.806824,0.806824,0.806824
498,498,None,201828,0,0,0,0,0,0,0,...,0,0,0,806823,0,0,806823,0.806824,0.806824,0.806824


In [72]:
# Plot matplotlib
plt.plot(model.t,model.S,label='S')
plt.plot(model.t,model.E,label='E')
plt.plot(model.t,model.I,label='I')
plt.plot(model.t,model.R,label='R')
plt.plot(model.t,model.H,label='H')
plt.plot(model.t,model.Iv,label='Iv')
plt.plot(model.t,model.D,label='D')
plt.xlim(0,300)
plt.legend(loc=0)
plt.title('SEIR from SEIRHVD lib')
plt.show()

In [73]:
%%capture
# Plot with pandas
seirconfig = 'cfg/SEIR.toml'
seirmodel = SEIR(config = seirconfig)# Input configuration file
seirmodel.integrate()

In [74]:
# Plot matplotlib
plt.plot(model.t,model.I,label='I_seirhvd')
plt.plot(seirmodel.t,seirmodel.I,label='I_seir')
plt.xlim(0,300)
plt.legend(loc=0)
plt.title('SEIR and SEIRHVD Comparison')
plt.show()

# Vaccines

In [105]:
# Vaccine campaign
Psi = cv19functions.Events(values=[0,20000,0],days=[[0,40],[40,60],[60,500]])

In [106]:
n = 100
plt.plot(np.arange(n),[Psi(i) for i in np.arange(n)])
plt.show()

In [114]:
%%capture
# Input configuration file
config = 'cfg/SEIRHVD_SEIR.toml'
# Build simulation object
model_vac = SEIRHVD(config = config,Psi=Psi,v = 1.0)
# Simulate (solve ODE)
model_vac.integrate()

In [115]:
v = list(np.arange(0,1,0.1))

In [116]:
model_vac2 = [SEIRHVD(config = config,Psi=Psi,v = i) for i in v]
for i in model_vac2:
    i.integrate()

In [117]:
np.linspace(0,1,len(v))

array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])

In [118]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(v)))

In [112]:
i = 0
for model_vac in model_vac2:
    #plt.plot(model_vac.t,model_vac.I,label='I with vaccines')
    plt.plot(model_vac.t,model_vac.I,label='I eff:'+str(model_vac.v(0)),color=colors[i])
    plt.plot(model_vac.t,model_vac.Iv,label='Iv eff:'+str(model_vac.v(0)),color=colors[i],linestyle='dashed')
    i+=1
plt.plot(np.arange(model_vac.tsim),[Psi(i) for i in np.arange(model_vac.tsim)])
plt.legend(loc=0)

plt.xlim(0,300)
plt.title('SEIRV')
plt.show() 

In [119]:
i = 0
for model_vac in model_vac2:
    #plt.plot(model_vac.t,model_vac.I,label='I with vaccines')
    plt.plot(model_vac.t,model_vac.H,label='H eff:'+str(model_vac.v(0)),color=colors[i])
    plt.plot(model_vac.t,model_vac.D,label='D eff:'+str(model_vac.v(0)),color=colors[i],linestyle='dashed')
    i+=1
plt.plot(np.arange(model_vac.tsim),[Psi(i) for i in np.arange(model_vac.tsim)])
plt.legend(loc=0)

plt.xlim(0,300)
plt.title('SEIRV')
plt.show() 

In [88]:
# Plot matplotlib

plt.plot(model_vac.t,model_vac.I,label='I with vaccines')
plt.plot(model.t,model.I,label='I no vaccines')
plt.plot(model_vac.t,model_vac.Iv,label='Vaccinated infected')
plt.plot(np.arange(model_vac.tsim),[Psi(i) for i in np.arange(model_vac.tsim)])
plt.xlim(0,300)
plt.legend(loc=0)
plt.title('SEIRV')
plt.show()

In [62]:
# Plot matplotlib
plt.plot(model_vac.t,model_vac.S,label='S')
plt.plot(model_vac.t,model_vac.E,label='Sv')
plt.xlim(0,300)
plt.legend(loc=0)
plt.title('SEIR from SEIRHVD lib')
plt.show()

# Hospitalization